In [35]:
import requests
import gradio as gr

def request_document_intelligence(image_path):
    # Azure Document Intelligence API 엔드포인트 및 인증 헤더 설정
    endpoint = "https://fimtrus-document-intelligence2.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read:analyze?_overload=analyzeDocument&api-version=2024-11-30"

    headers = {
        "Ocp-Apim-Subscription-Key": "1WBG3li4tMyQRvo2cQOy0bPHkkqXMBMKwDJeNB1ZKTkUSchWSYOsJQQJ99BFACYeBjFXJ3w3AAALACOGO2DP",
        "Content-Type": "image/png"
    }

    # 이미지 파일을 바이너리로 읽어서 POST 요청으로 전송
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
    
    response = requests.post(endpoint, headers=headers, data=image_data)
    
    # 정상적으로 요청이 처리되지 않은 경우 에러 출력 후 None 반환
    if response.status_code != 202:
        print(f"Error: {response.status_code}")
        print("Error: {}".format(response.status_code))
        return None

    # 응답 헤더에서 Operation-Location(비동기 결과 조회 URL) 추출
    url = response.headers['Operation-Location']
    print(url)
    
    # 결과가 준비될 때까지 반복적으로 상태 확인
    while True:    
        result_response = requests.get(url, headers=headers)

        if result_response.status_code != 200:
            print(f"Error: {result_response.status_code}")
            return None

        result_response_json = result_response.json()
        current_status = result_response_json.get("status")

        # 분석이 아직 진행 중이면 계속 대기
        if current_status == "running":
            print("Current status: {}".format(current_status))
            continue
        else:
            break
    
    # 분석이 성공적으로 끝난 경우 결과 반환, 아니면 None 반환
    if current_status == "succeeded":
        return result_response_json
    else:
        return None

def random_color():
    # 랜덤한 RGB 색상 튜플 반환
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 

def get_font():
    # OS별로 적절한 폰트 객체 반환 (한글 지원)
    from PIL import ImageFont
    import platform
    
    font_size = 20
    
    try:
        if platform.system() == "Windows":
            return ImageFont.truetype("malgun.ttf", font_size)
        elif platform.system() == "Darwin":  # macOS
            return ImageFont.truetype("AppleGothic.ttf", font_size)
        else:  # Linux      
            return ImageFont.load_default(size=font_size)
    except IOError:
        # 폰트 파일이 없을 경우 기본 폰트 사용
        return ImageFont.load_default(size=font_size)

def draw_polygon(image_path, response_data):
    # 분석 결과의 각 문단(Paragraph)에 대해 폴리곤과 텍스트를 이미지에 그림
    from PIL import Image, ImageDraw
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    block_list = response_data['analyzeResult']['paragraphs']
    
    for block in block_list:
        line_color = random_color()  # 각 블록마다 랜덤 색상 지정
        font = get_font()
        # 폴리곤 좌표 추출
        polygon = block['boundingRegions'][0]['polygon']
        # 텍스트 내용 추출
        content = block['content']
        # (x, y) 튜플 리스트로 변환
        polygon_points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]

        # 폴리곤 그리기
        draw.polygon(polygon_points, outline=line_color, width=2)
        # 폴리곤 시작점 위에 텍스트 표시
        draw.text((polygon[0], polygon[1] - 20), content, fill=line_color, font=font)
        
    return image

# Gradio UI 구성
with gr.Blocks() as demo:
    
    def change_image(image_path):
        # 이미지가 변경될 때마다 분석 요청 및 결과 이미지 반환
        response_data = request_document_intelligence(image_path)
        image = draw_polygon(image_path, response_data)
        return image
    
    # 입력 이미지와 출력 이미지를 나란히 배치
    with gr.Row():
        input_image = gr.Image(label="이미지 선택", type="filepath", width=500)
        output_image = gr.Image(label="결과 이미지", type="pil", interactive=False, width=500)
    
    # 입력 이미지가 변경될 때 change_image 함수 호출
    input_image.change(change_image, inputs=[input_image], outputs=[output_image])
    
demo.launch()

# change_image('/Users/fimtrus/Downloads/카드뉴스_한글날_1_1.jpg')

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


https://fimtrus-document-intelligence2.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-read/analyzeResults/fe005cd0-c6a9-4ceb-b8a5-7e879a4dc341?api-version=2024-11-30


In [10]:
import gradio as gr